In [14]:
from abc import ABC, abstractmethod
from numerapi import CryptoAPI
import pandas as pd
import numpy as np


# Generic Oracle Interface
class OracleInterface(ABC):
    @abstractmethod
    def fetch_portfolio_weights(self, timestamp: str) -> dict:
        """Fetch portfolio weights for the given timestamp."""
        pass

    @abstractmethod
    def validate_weights(self, weights: dict) -> bool:
        """Validate the portfolio weights."""
        pass

# Oracle Implementation
class ExampleOracle(OracleInterface):
    def fetch_portfolio_weights(self, timestamp: str) -> dict:
        api = CryptoAPI()
        api.download_dataset(
        	"crypto/v1.0/historical_meta_models.csv",
        	"historical_meta_models.csv"
        )
        
        #load historical MM
        mm = pd.read_csv('historical_meta_models.csv')
        mm['date'] = pd.to_datetime( mm['date'] )

        x = mm.loc[ mm['date'] == sorted( mm['date'] )[-1], ['symbol','meta_model'] ]
        x['w'] = ( x['meta_model'].rank(pct=True) - 0.5 )
        x['w'] = x['w'] / x['w'].abs().sum()
        
        print(f"[Oracle] Computing portfolio weights for timestamp {timestamp}.")
        weights = dict(zip(x["symbol"], x["w"]))
        return weights

    def validate_weights(self, weights: dict) -> bool:
        # Ensure weights sum to 1
        valid = (np.abs( np.array( [ weights[k] for k in weights.keys() ] ) ).sum() - 1.0) < 1e-6
        if not valid:
            print("[Oracle] Weights validation failed.")
        return valid

# Portfolio Manager
class PortfolioManager:
    def __init__(self, oracle: OracleInterface):
        """Initialize the Portfolio Manager with an Oracle implementation."""
        self.oracle = oracle

    def manage_portfolio(self, timestamp: str):
        """Fetch weights from the Oracle and print them."""
        print(f"[PortfolioManager] Requesting portfolio weights for {timestamp}.")
        weights = self.oracle.fetch_portfolio_weights(timestamp)

        if not self.oracle.validate_weights(weights):
            raise ValueError("Invalid portfolio weights received from Oracle.")

        print(f"[PortfolioManager] Portfolio weights: {weights}")

# Main Execution
if __name__ == "__main__":
    # Instantiate Oracle
    oracle = ExampleOracle()

    # Instantiate Portfolio Manager with Oracle
    portfolio_manager = PortfolioManager(oracle)

    # Perform portfolio management
    portfolio_manager.manage_portfolio(timestamp="2024-12-13T10:00:00Z")


[PortfolioManager] Requesting portfolio weights for 2024-12-13T10:00:00Z.


2024-12-13 11:37:57,230 INFO numerapi.utils: target file already exists
2024-12-13 11:37:57,231 INFO numerapi.utils: download complete


[Oracle] Computing portfolio weights for timestamp 2024-12-13T10:00:00Z.
[PortfolioManager] Portfolio weights: {'BTC': 0.00392, 'ETH': 0.003984, 'NMR': 0.0026399999999999996, 'XRP': -0.003104, 'SOL': 0.002912, 'BNB': 0.004, 'DOGE': -0.0016960000000000002, 'ADA': -0.002016, 'TRX': 0.0007839999999999998, 'AVAX': 0.000976, 'LINK': -0.0007519999999999997, 'SHIB': 0.0032960000000000003, 'TON': 0.0035679999999999996, 'DOT': -0.0012639999999999997, 'SUI': -0.001952, 'XLM': -0.00304, 'HBAR': -0.0036, 'BCH': 0.0027519999999999997, 'UNI': -0.001808, 'PEPE': -0.00017600000000000016, 'LTC': 0.0014240000000000004, 'NEAR': 0.001008, 'APT': 0.0013920000000000004, 'ICP': 0.001952, 'AAVE': -0.00025599999999999977, 'POL': -0.00248, 'ETC': 0.002416, 'RENDER': -0.00022400000000000019, 'CRO': -0.001008, 'VET': -0.0022080000000000003, 'BGB': -0.002144, 'FET': 0.0024800000000000004, 'MNT': -0.000992, 'TAO': -0.00011200000000000009, 'ARB': 6.400000000000005e-05, 'KAS': 0.0017119999999999998, 'FIL': -0.0008799